# Output will be cropped images of every glomeruli in a zipfile....
## Rember to run 3rd only once
## the output images will help you to train model faster....
[Link to notebook for rle2mask and a train_test.csv file](https://www.kaggle.com/realc9der/ecoding-of-glomeruli/edit)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import cv2 as cv
import tifffile
import os

In [ ]:
"""
os.mkdir("../working/images")
"""

In [ ]:
# a cell that use all the above function and variable to create a the image cropping
def get_id(n):
    return df_train.id[n]


def Coor_of_Glomeruli(ID,n):
    df = pd.read_json("../input/hubmap-kidney-segmentation/train/"+ID+".json")
    return np.array(df.geometry[n]["coordinates"])[0,:,:],n


def get_image(ID):
    return tifffile.imread("../input/hubmap-kidney-segmentation/train/"+ID+".tiff")


def find_box(coor_array,h,w):
    L_x = coor_array[0][0]
    S_x = L_x
    L_y = coor_array[0][1]
    S_y = L_y
    for i in range(len(coor_array)):
        if L_x < coor_array[i][0]:
            L_x = coor_array[i][0]
    for i in range(len(coor_array)):
        if S_x > coor_array[i][0]:
            S_x = coor_array[i][0]
    for i in range(len(coor_array)):
        if L_y < coor_array[i][1]:
            L_y = coor_array[i][1]
    for i in range(len(coor_array)):
        if S_y > coor_array[i][1]:
            S_y = coor_arraty[i][1]
    return L_x,S_x,L_y,S_y


def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 



def save_image(sub_image,image_id,no_of_glomeruli):
    path = '../working/images/'+image_id+"_"+str(no_of_glomeruli)+'.png'
    #ther many glomeruli in image so no_of_glomeruli represent that 
    cv.imwrite(path,sub_image)

    
def total_glomeruli(ID):
    df = pd.read_json("../input/hubmap-kidney-segmentation/train/"+ID+".json")
    return len(df.geometry)



def rle2mask(mask_rle,shape):
    s = mask_rle.split()
    starts, lengths = [
    np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])
    ]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape).T

########################
########################
df_train = pd.read_csv("../input/hubmap-kidney-segmentation/train.csv")
train_data = pd.DataFrame(data=None,columns=['file_name','rle',],index=None) # creating a empyt data

path = os.path.join("../working/images")
#os.mkdir(path,)
########################
########################





In [ ]:
for i in range(len(df_train.id)):
    image_id = get_id(i) #got the image id 
    #get the coordinates array of each glomeruli
    image = tifffile.imread("../input/hubmap-kidney-segmentation/train/"+image_id+".tiff")#get_image(image_id) #image
    Total_Glomeruli = total_glomeruli(image_id)
    w,h = image.shape[:2]
    path = os.path.join("../working/cropped_images/")
#cropping images
    for j in range(Total_Glomeruli):
        coor_array,no_of_Glomeruli = Coor_of_Glomeruli(image_id,j)
        #get the box
        L_x,S_x,L_y,S_y = find_box(coor_array,h,w)
        #crop the image
        if S_x < w and S_y < h and L_x < w and L_y < h:
            sub_image = image[S_y:L_y,S_x:L_x]    
        # save the cropped image
        save_image(sub_image,image_id,no_of_Glomeruli)
        #start with new image

    print("done Id ",image_id)

# Now main task is to create a zip file with all the images


In [ ]:
!zip -r cropped_images.zip ../working/images/

In [ ]:
print("go download")